# Stage 10a: Regression Modeling for Bike Demand Prediction

## Overview
Comprehensive regression modeling approach for bike demand prediction with time-aware splitting, feature selection, assumption validation, and risk-aware interpretation.

## Modeling Approach: Regression
**Rationale**: Bike demand is continuous numerical target, making regression most appropriate for predicting exact rental counts.

In [ ]:
# Import libraries
import sys
sys.path.append('../src')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.feature_selection import SelectKBest, f_regression
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8')
print("🤖 Libraries Loaded")

In [ ]:
# Load data
try:
    data = pd.read_csv('../data/processed/engineered_features.csv')
except FileNotFoundError:
    from feature_engineering import feature_engineering_pipeline
    raw_data = pd.read_csv('../data/sample-data.csv')
    data = feature_engineering_pipeline(raw_data)
    data.to_csv('../data/processed/engineered_features.csv', index=False)

print(f"📊 Dataset: {data.shape}")
print(f"Target range: {data['demand'].min()}-{data['demand'].max()}")

## 1. Feature Selection & Preprocessing

In [ ]:
# Remove target leakage features
y = data['demand'].copy()
leakage_features = ['demand', 'demand_rolling_3h', 'demand_rolling_6h', 
                   'demand_lag_1', 'demand_lag_2', 'demand_change']
X = data.drop(columns=[col for col in leakage_features if col in data.columns])

print(f"🎯 Target: demand")
print(f"🚫 Removed {len([col for col in leakage_features if col in data.columns])} leakage features")
print(f"📊 Features: {X.shape[1]}")

# Handle missing values
if X.isnull().sum().sum() > 0:
    X = X.fillna(X.median())
    print("⚠️ Filled missing values")
else:
    print("✅ No missing values")

In [ ]:
# Feature selection - top 15 features
k_best = min(15, X.shape[1])
selector = SelectKBest(score_func=f_regression, k=k_best)
X_selected = selector.fit_transform(X, y)
selected_features = X.columns[selector.get_support()].tolist()

print(f"🏆 Selected {k_best} top features:")
feature_scores = pd.DataFrame({
    'feature': X.columns,
    'f_score': selector.scores_,
    'selected': selector.get_support()
}).sort_values('f_score', ascending=False)

print(feature_scores.head(10))
X = X[selected_features]

## 2. Time-Aware Train-Test Split

In [ ]:
# Time-aware split (80/20)
n_samples = len(X)
train_size = int(0.8 * n_samples)

X_train = X.iloc[:train_size].copy()
X_test = X.iloc[train_size:].copy()
y_train = y.iloc[:train_size].copy()
y_test = y.iloc[train_size:].copy()

print(f"📊 Time-Aware Split:")
print(f"   Training: {X_train.shape[0]} samples ({X_train.shape[0]/n_samples*100:.1f}%)")
print(f"   Test: {X_test.shape[0]} samples ({X_test.shape[0]/n_samples*100:.1f}%)")
print(f"   No temporal overlap: {X_train.index.max() < X_test.index.min()}")

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
print("📏 Features scaled (StandardScaler)")

## 3. Model Training & Evaluation

In [ ]:
# Define models
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(alpha=1.0, random_state=42),
    'Lasso Regression': Lasso(alpha=0.1, random_state=42),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42)
}

# Train and evaluate
results = {}
trained_models = {}

print("🤖 Training Models:")
for name, model in models.items():
    # Use scaled features for linear models
    if 'Forest' in name:
        X_tr, X_te = X_train, X_test
    else:
        X_tr, X_te = X_train_scaled, X_test_scaled
    
    model.fit(X_tr, y_train)
    trained_models[name] = model
    
    y_pred_train = model.predict(X_tr)
    y_pred_test = model.predict(X_te)
    
    results[name] = {
        'train_r2': r2_score(y_train, y_pred_train),
        'test_r2': r2_score(y_test, y_pred_test),
        'test_rmse': np.sqrt(mean_squared_error(y_test, y_pred_test)),
        'test_mae': mean_absolute_error(y_test, y_pred_test),
        'predictions': y_pred_test
    }
    
    print(f"   {name}: R²={results[name]['test_r2']:.3f}, RMSE={results[name]['test_rmse']:.2f}")

# Best model
best_model_name = max(results.keys(), key=lambda k: results[k]['test_r2'])
print(f"\n🏆 Best Model: {best_model_name} (R²={results[best_model_name]['test_r2']:.3f})")

## 4. Diagnostic Plots & Assumption Validation

In [ ]:
# Focus on best linear model for diagnostics
linear_models = ['Linear Regression', 'Ridge Regression', 'Lasso Regression']
best_linear = max([m for m in linear_models if m in results], 
                 key=lambda k: results[k]['test_r2'])

model = trained_models[best_linear]
y_pred = results[best_linear]['predictions']
residuals = y_test - y_pred

print(f"🔍 Diagnostic Analysis: {best_linear}")
print(f"   Residual mean: {residuals.mean():.6f} (should be ~0)")
print(f"   Residual std: {residuals.std():.3f}")

# Diagnostic plots
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Residuals vs Fitted
axes[0,0].scatter(y_pred, residuals, alpha=0.7)
axes[0,0].axhline(y=0, color='red', linestyle='--')
axes[0,0].set_title('Residuals vs Fitted')
axes[0,0].set_xlabel('Fitted Values')
axes[0,0].set_ylabel('Residuals')

# Q-Q Plot
stats.probplot(residuals, dist="norm", plot=axes[0,1])
axes[0,1].set_title('Q-Q Plot (Normality Check)')

# Residuals histogram
axes[1,0].hist(residuals, bins=10, alpha=0.7, edgecolor='black')
axes[1,0].set_title('Residuals Distribution')
axes[1,0].set_xlabel('Residuals')

# Actual vs Predicted
axes[1,1].scatter(y_test, y_pred, alpha=0.7)
axes[1,1].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
axes[1,1].set_title('Actual vs Predicted')
axes[1,1].set_xlabel('Actual')
axes[1,1].set_ylabel('Predicted')

plt.tight_layout()
plt.show()

## 5. Feature Importance & Coefficients Analysis

In [ ]:
# Analyze coefficients for best linear model
if hasattr(trained_models[best_linear], 'coef_'):
    coefficients = pd.DataFrame({
        'feature': selected_features,
        'coefficient': trained_models[best_linear].coef_
    }).sort_values('coefficient', key=abs, ascending=False)
    
    print(f"🔍 {best_linear} Coefficients:")
    print(coefficients.head(10))
    
    # Plot top coefficients
    plt.figure(figsize=(10, 6))
    top_coef = coefficients.head(10)
    colors = ['red' if x < 0 else 'blue' for x in top_coef['coefficient']]
    plt.barh(range(len(top_coef)), top_coef['coefficient'], color=colors, alpha=0.7)
    plt.yticks(range(len(top_coef)), top_coef['feature'])
    plt.xlabel('Coefficient Value')
    plt.title(f'Top 10 Feature Coefficients - {best_linear}')
    plt.axvline(x=0, color='black', linestyle='-', alpha=0.3)
    plt.tight_layout()
    plt.show()

# Random Forest feature importance
if 'Random Forest' in trained_models:
    rf_importance = pd.DataFrame({
        'feature': selected_features,
        'importance': trained_models['Random Forest'].feature_importances_
    }).sort_values('importance', ascending=False)
    
    print(f"\n🌳 Random Forest Feature Importance:")
    print(rf_importance.head(10))

## 6. Model Interpretation & Risk Assessment

In [ ]:
# Model performance summary
performance_df = pd.DataFrame({
    'Model': list(results.keys()),
    'Test_R2': [results[m]['test_r2'] for m in results.keys()],
    'Test_RMSE': [results[m]['test_rmse'] for m in results.keys()],
    'Test_MAE': [results[m]['test_mae'] for m in results.keys()],
    'Overfitting': [results[m]['train_r2'] - results[m]['test_r2'] for m in results.keys()]
}).sort_values('Test_R2', ascending=False)

print("📊 Model Performance Summary:")
print(performance_df.round(3))

print(f"\n🎯 Key Insights:")
print(f"   • Best model explains {results[best_model_name]['test_r2']:.1%} of demand variance")
print(f"   • Average prediction error: {results[best_model_name]['test_rmse']:.1f} bikes")
print(f"   • Mean absolute error: {results[best_model_name]['test_mae']:.1f} bikes")

# Risk assessment
print(f"\n⚠️ Risk Assessment:")
max_error = abs(residuals).max()
print(f"   • Maximum prediction error: {max_error:.1f} bikes")
print(f"   • 95% of errors within: ±{np.percentile(abs(residuals), 95):.1f} bikes")
print(f"   • Model reliability: {'High' if results[best_model_name]['test_r2'] > 0.8 else 'Moderate' if results[best_model_name]['test_r2'] > 0.6 else 'Low'}")

# Business interpretation
demand_range = y.max() - y.min()
relative_error = results[best_model_name]['test_rmse'] / demand_range
print(f"   • Relative error: {relative_error:.1%} of demand range")
print(f"   • Suitable for: {'Operational planning' if relative_error < 0.15 else 'Strategic planning only'}")

## 7. Save Results

In [ ]:
# Save model results
import os
from datetime import datetime

os.makedirs('../data/processed', exist_ok=True)

# Save predictions
predictions_df = pd.DataFrame({
    'actual': y_test,
    'predicted': results[best_model_name]['predictions'],
    'residuals': residuals
})
predictions_df.to_csv('../data/processed/model_predictions.csv', index=False)

# Save model summary
performance_df.to_csv('../data/processed/model_performance.csv', index=False)

print("💾 Results saved:")
print("   • Model predictions: ../data/processed/model_predictions.csv")
print("   • Performance summary: ../data/processed/model_performance.csv")
print(f"\n🎯 Modeling Complete! Best model: {best_model_name} (R²={results[best_model_name]['test_r2']:.3f})")